In [2]:
import sagemaker
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

import import_ipynb
import c2_training_step

def create_post_processing_step(training_step):
    
    xgboost_container_uri = sagemaker.image_uris.retrieve("xgboost", 'eu-west-1', "1.7-1")

    evaluation_processor = ScriptProcessor(
        image_uri=xgboost_container_uri,
        command=["python3"],
        instance_type='ml.m5.large',
        instance_count=1,
        base_job_name="evaluate",
        role=sagemaker.get_execution_role(),
    )
    
    # define inputs
    model_processing_input = ProcessingInput(source=training_step.properties.ModelArtifacts.S3ModelArtifacts, destination="/opt/ml/processing/model")
    data_processing_input = ProcessingInput(source='s3://ktzouvan-trading-point-sagemaker-poc/datasets/marketing/', destination="/opt/ml/processing/test")
    
    # define output
    evaluation_processing_output = ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=training_step.properties.ModelArtifacts.S3ModelArtifacts)

    evaluation_report_file = PropertyFile(
        name="EvaluationReport",
        output_name="evaluation",
        path="0_evaluation.json"
    )    
    evaluate_step = ProcessingStep(
        name="Evaluate_Model",
        processor=evaluation_processor,
        inputs=[model_processing_input, data_processing_input],
        outputs=[evaluation_processing_output],
        code="d1_post_processing.py",
        job_arguments =['--model_path', '/opt/ml/processing/model','--model_file', 'model.tar.gz','--data_path', '/opt/ml/processing/test', '--output_path', '/opt/ml/processing/evaluation'],
        property_files=[evaluation_report_file]
    )

    return evaluate_step